In [1]:
# import dependencies

import csv
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load in CSVs

df1_battles = pd.read_csv("Raw Datasets/battles.csv")
df2_character_deaths = pd.read_csv("Raw Datasets/character-deaths.csv")
df3_character_predictions = pd.read_csv("Raw Datasets/character-predictions.csv")

## Table 1 - Members of each House

In [3]:
# Change column 'Name' of df2_character_deaths to 'name' so that we can merge df2 and df3
df2_character_deaths = df2_character_deaths.rename(columns={"Name": "name"})

# Merge tables on character names
merged_table = pd.merge(df2_character_deaths, df3_character_predictions, on="name", how="outer")

# Drop duplicate values
merged_table.drop_duplicates('name')

# Display merged table 
merged_table

name      Allegiances  Death Year  \
0                        Addam Marbrand        Lannister         NaN   
1               Aegon Frey (Jinglebell)             None       299.0   
2                       Aegon Targaryen  House Targaryen         NaN   
3                         Adrack Humble    House Greyjoy       300.0   
4                        Aemon Costayne        Lannister         NaN   
5                       Aemon Estermont        Baratheon         NaN   
6     Aemon Targaryen (son of Maekar I)    Night's Watch       300.0   
7                            Aenys Frey             None       300.0   
8                         Aeron Greyjoy    House Greyjoy         NaN   
9                                Aethan    Night's Watch         NaN   
10                                Aggar    House Greyjoy       299.0   
11                                 Aggo  House Targaryen         NaN   
12                        Alan of Rosby    Night's Watch       300.0   
13                              Alayaya             None         NaN   
14                          Albar Royce            Arryn         NaN   
15                               Albett    Night's Watch         NaN   
16                             Alebelly      House Stark       299.0   
17                     Alerie Hightower     House Tyrell         NaN   
18                   Alesander Staedmon        Baratheon         NaN   
19                      Alester Florent        Baratheon       300.0   
20                      Alia of Braavos             None         NaN   
21                          Alla Tyrell     House Tyrell         NaN   
22                      Allard Seaworth        Baratheon       299.0   
23                       Alliser Thorne    Night's Watch         NaN   
24                                 Alyn      House Stark       298.0   
25                         Alyn Ambrose           Tyrell         NaN   
26                       Alyn Estermont        Baratheon         NaN   
27                      Alyn Stackspear        Lannister         NaN   
28                        Alys Karstark            Stark         NaN   
29                      Alysane Mormont            Stark         NaN   
...                                 ...              ...         ...   
1984                      Willam Dustin              NaN         NaN   
1985                   Tyland Lannister              NaN         NaN   
1986                      Tyrion Tanner              NaN         NaN   
1987                   Damion Lannister              NaN         NaN   
1988                   Lyonel Baratheon              NaN         NaN   
1989                      Tya Lannister              NaN         NaN   
1990                             Rhaego              NaN         NaN   
1991                      Samwell Stone              NaN         NaN   
1992                     Rhae Targaryen              NaN         NaN   
1993                          Togg Joth              NaN         NaN   
1994                           Tanselle              NaN         NaN   
1995                  Rhaelle Targaryen              NaN         NaN   
1996                   Maelor Targaryen              NaN         NaN   
1997                 Alysanne Targaryen              NaN         NaN   
1998                    Baela Targaryen              NaN         NaN   
1999                      Brandon Stark              NaN         NaN   
2000                      Jafer Flowers              NaN         NaN   
2001                            Rossart              NaN         NaN   
2002                Daemon II Blackfyre              NaN         NaN   
2003                        Hugh Hammer              NaN         NaN   
2004                   Tygett Lannister              NaN         NaN   
2005                  Rhaegar Targaryen              NaN         NaN   
2006                     Bethany Bolton              NaN         NaN   
2007                              Tarle              NaN         NaN   
200

In [4]:
# Group the table by the different Houses and list those characters that belong to the specific house
Houses = merged_table.groupby(["Allegiances", 'name'])["name"].unique()

# Convert to DataFrame
members_Of_Each_House = pd.DataFrame(Houses)
members_Of_Each_House.columns = ["X"]
members_Of_Each_House.reset_index(level=[0,1], inplace=True)

# Drop the "X" column
members_Of_Each_House.drop("X", axis=1, inplace = True)
members_Of_Each_House.head()

Allegiances             name
0       Arryn      Albar Royce
1       Arryn      Andar Royce
2       Arryn    Anya Waynwood
3       Arryn  Benedar Belmore
4       Arryn            Byron

In [5]:
members_Of_Each_House.reset_index(level=[0], inplace=True)
members_Of_Each_House.head()

cols = ['index', 'Allegiances', 'name']
members_Of_Each_House = members_Of_Each_House[cols].copy()

# # Rename column headers
members_Of_Each_House = members_Of_Each_House.rename(columns={"index": "id", "Allegiances": "allegiance", "name": "character_name"})

# # Set index
members_Of_Each_House.set_index(["id"], inplace=True)

members_Of_Each_House.head()

allegiance   character_name
id                            
0       Arryn      Albar Royce
1       Arryn      Andar Royce
2       Arryn    Anya Waynwood
3       Arryn  Benedar Belmore
4       Arryn            Byron

## Table 2 - Find win/loss ratio per each House

In [6]:
# Determine the number of battles that the attacker was the winner. 

lannisters_win_count = 0
stark_win_count = 0
greyjoy_win_count = 0
bolton_win_count = 0
baratheon_win_count = 0
darry_win_count = 0
frey_win_count = 0
free_folk_win_count = 0
brave_Companions_win_count = 0
brotherhood_without_Banners = 0
bracken_win_count = 0

for index, row in df1_battles.iterrows():
    if row['attacker_1'] == "Lannister" and row['attacker_outcome'] == 'win':
        lannisters_win_count += 1
    elif row['attacker_1'] == "Stark" and row['attacker_outcome'] == 'win':
        stark_win_count += 1
    elif row['attacker_1'] == "Greyjoy" and row['attacker_outcome'] == 'win':
        greyjoy_win_count += 1
    elif row['attacker_1'] == "Bolton" and row['attacker_outcome'] == 'win':
        bolton_win_count += 1
    elif row['attacker_1'] == "Baratheon" and row['attacker_outcome'] == 'win':
        baratheon_win_count += 1
    elif row['attacker_1'] == "Darry" and row['attacker_outcome'] == 'win':
        darry_win_count += 1
    elif row['attacker_1'] == "Brotherhood without Banners" and row['attacker_outcome'] == 'win':
        brotherhood_without_Banners += 1
    elif row['attacker_1'] == "Frey" and row['attacker_outcome'] == 'win':
        frey_win_count += 1
    elif row['attacker_1'] == "Free folk" and row['attacker_outcome'] == 'win':
        free_folk_win_count += 1
    elif row['attacker_1'] == "Brave Companions" and row['attacker_outcome'] == 'win':
        brave_Companions_win_count += 1
    else:
        bracken_win_count += 1

In [7]:
# Determine the number of battles each house has been the primary attacker

l_total_battles = 0 
s_total_battles = 0 
g_total_battles = 0 
bolt_total_battles = 0 
bara_total_battles = 0 
darry_total_battles = 0 
frey_total_battles = 0 
free_total_battles = 0 
brave_total_battles = 0 
brother_total_battles = 0 
bracken_total_battles = 0 

for index, row in df1_battles.iterrows():
    if row['attacker_1'] == "Lannister":
        l_total_battles+= 1
    elif row['attacker_1'] == "Stark":
        s_total_battles += 1
    elif row['attacker_1'] == "Greyjoy":
        g_total_battles += 1
    elif row['attacker_1'] == "Bolton":
         bolt_total_battles+= 1
    elif row['attacker_1'] == "Baratheon":
        bara_total_battles += 1
    elif row['attacker_1'] == "Darry":
        darry_total_battles += 1
    elif row['attacker_1'] == "Brotherhood without Banners":
        brother_total_battles += 1
    elif row['attacker_1'] == "Frey":
        frey_total_battles += 1
    elif row['attacker_1'] == "Free folk":
        free_total_battles += 1
    elif row['attacker_1'] == "Brave Companions":
        brave_total_battles += 1
    else:
        bracken_total_battles += 1

In [8]:
# Calculate Win Ratio for each House

lannister_ratio = lannisters_win_count / l_total_battles
stark_ratio =  stark_win_count / s_total_battles
greyjoy_ratio = greyjoy_win_count / g_total_battles
bolton_ratio = bolton_win_count / bolt_total_battles
baratheon_ratio = baratheon_win_count / bara_total_battles
darry_ratio = darry_win_count / darry_total_battles
brother_ratio = brotherhood_without_Banners / brother_total_battles
frey_ratio = frey_win_count / frey_total_battles
free_folk_ratio = free_folk_win_count / free_total_battles
brave_ratio = brave_Companions_win_count / brave_total_battles
bracken_ratio = bracken_win_count / bracken_total_battles

In [9]:
# Create a dataframe that includes the number of battles that each has started and their win ratio in these battles. 

ratios_df = pd.DataFrame({"House": ['Lannister', 'Stark', 'Greyjoy', 'Bolton', 'Baratheon', 'Darry', 'Brotherhood W/O Banners',
                                    'Frey', 'Free Folk', 'Brave Companions', 'Bracken'],"Total Battles": [l_total_battles, s_total_battles,
                                                                                                          g_total_battles, bolt_total_battles, bara_total_battles, darry_total_battles, brother_total_battles, frey_total_battles, free_total_battles, brave_total_battles, bracken_total_battles],
                           'Win Ratio': [lannister_ratio, stark_ratio, greyjoy_ratio, bolton_ratio, baratheon_ratio, 
                                        darry_ratio, brother_ratio, frey_ratio, free_folk_ratio, brave_ratio, bracken_ratio]})

ratios_df

House  Total Battles  Win Ratio
0                 Lannister              8   0.875000
1                     Stark              8   0.750000
2                   Greyjoy              7   1.000000
3                    Bolton              2   1.000000
4                 Baratheon              6   0.666667
5                     Darry              1   1.000000
6   Brotherhood W/O Banners              1   1.000000
7                      Frey              2   1.000000
8                 Free Folk              1   0.000000
9          Brave Companions              1   1.000000
10                  Bracken              1   7.000000

In [10]:
ratios_df.reset_index(level=[0], inplace=True)

cols = ['index', 'House', 'Total Battles', 'Win Ratio']
ratios_df = ratios_df[cols].copy()

# Rename column headers
ratios_df = ratios_df.rename(columns={"index": "id", "House": "allegiance", "Total Battles": "total_battles", 
                                    "Win Ratio": "win_ratio"})

# Set index
ratios_df.set_index(["id"], inplace=True)

ratios_df.head()

allegiance  total_battles  win_ratio
id                                     
0   Lannister              8   0.875000
1       Stark              8   0.750000
2     Greyjoy              7   1.000000
3      Bolton              2   1.000000
4   Baratheon              6   0.666667

## Table 3 - List of Members that remain alive per House

In [11]:
df3_character_predictions.drop(['S.No','actual','pred','alive','plod','title', 'male','culture', 'dateOfBirth', 'DateoFdeath',
 'mother', 'father', 'heir', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5', 'isAliveMother', 'isAliveFather', 'isAliveHeir',
 'isAliveSpouse', 'isMarried', 'isNoble', 'age', 'numDeadRelations', 'boolDeadRelations', 'isPopular', 'popularity'], axis = 1, inplace = True)

In [12]:
# Retrieve only those characters that remain alive
df3_character_predictions[df3_character_predictions.isAlive != 0]

name                              house  isAlive
1             Walder Frey                         House Frey        1
2            Addison Hill                        House Swyft        1
4          Sylva Santagar                     House Santagar        1
5        Tommen Baratheon                                NaN        1
9          Wilbert Osgrey                       House Osgrey        1
11          Will (orphan)                                NaN        1
12          Will (squire)                                NaN        1
13       Will (Standfast)                       House Osgrey        1
14            Will (Treb)                       House Osgrey        1
15            Will Humble                       House Humble        1
16                 Willam                                NaN        1
17           Willem Wylde                        House Wylde        1
18               Willifer                                NaN        1
19                 Willit                        House Swyft        1
20            Willis Wode                         House Wode        1
21            Willis Fell                         House Fell        1
22          Willow Heddle        Brotherhood Without Banners        1
23                 Willum                                NaN        1
24                 Wolmer                       House Webber        1
27                  Wulfe                      House Greyjoy        1
28       Wyl the Whittler                                NaN        1
31    Wun Weg Wun Dar Wun                                NaN        1
32                  Wylla                        House Dayne        1
33      Wynafryd Manderly                     House Manderly        1
34                 Xhondo                                NaN        1
35           Ygon Farwynd  House Farwynd of the Lonely Light        1
36                    Yna                         Happy Port        1
37           Yohn Farwynd  House Farwynd of the Lonely Light        1
38                 Yorkel                    House Targaryen        1
39            Yorko Terys                                NaN        1
...                   ...                                ...      ...
1895                Satin                      Night's Watch        1
1896     Xaro Xhoan Daxos                           Thirteen        1
1898      Arianne Martell                      House Martell        1
1899   Damon Dance-for-Me                       House Bolton        1
1903        Dunstan Drumm                        House Drumm        1
1904          Daemon Sand                     House Allyrion        1
1906          Lysono Maar                     Golden Company        1
1908     Joyeuse Erenford                     House Erenford        1
1910           Tyene Sand                      House Martell        1
1914              Chataya                  Chataya's brothel        1
1916     Tygett Lannister                    House Lannister        1
1917         Sarella Sand                      House Martell        1
1920               Dunsen                      House Clegane        1
1922       Bethany Bolton                       House Bolton        1
1923        Desmond Grell                        House Grell        1
1924       Devan Seaworth                     House Seaworth        1
1925               Gendry        brotherhood without banners        1
1926     Gran Goodbrother                  House Goodbrother        1
1927         Loras Tyrell                       House Tyrell        1
1928              Ricasso                      House Martell        1
1930           Yohn Royce                        House Royce        1
1931               Yandry                    House Targaryen        1
1932                Tarle                        Drowned men        1
1933                Temmo                                NaN        1
1935  Gormond Goodbrother                  House Goodbrother        1
1936        Walder Rivers      

In [13]:
members_alive = df3_character_predictions.groupby(['house', 'name'])['name'].unique()

In [14]:
# Convert to DataFrame
new_members_alive = pd.DataFrame(members_alive)
new_members_alive.columns = ["X"]
new_members_alive.reset_index(level=[0,1], inplace=True)

# # Drop the "X" column
new_members_alive.drop("X", axis=1, inplace = True)

new_members_alive.reset_index(level=[0], inplace=True)

cols = ['index', 'house', 'name']
members_still_alive_transformed = new_members_alive[cols].copy()

# Rename column headers
members_still_alive_transformed = members_still_alive_transformed.rename(columns={"index": "id",
                                                         "house": "allegiance",
                                                         "name": "character_name"})

# Set index
members_still_alive_transformed.set_index(["id"], inplace=True)

members_still_alive_transformed.head()

allegiance character_name
id                                  
0   Alchemists' Guild          Belis
1   Alchemists' Guild        Garigus
2   Alchemists' Guild        Hallyne
3   Alchemists' Guild       Malliard
4   Alchemists' Guild       Munciter

## Create Database Connection

In [19]:
tempPass = input('Enter your name SQL Password ')

connection_string = "root:" + str(tempPass) + "@localhost/GoT_db"
engine = create_engine(f'mysql://{connection_string}')

Enter your name SQL Password Ilovesoccer3


In [20]:
# Confirm tables
engine.table_names()

['members_Of_Each_House', 'members_alive', 'ratios']

## Load DataFrames into Tables

In [21]:
members_still_alive_transformed.to_sql(name='members_alive', con=engine, if_exists='append', index=True)

In [22]:
ratios_df.to_sql(name='ratios', con=engine, if_exists='append', index=True)

In [23]:
members_Of_Each_House.to_sql(name='members_Of_Each_House', con=engine, if_exists='append', index=True)